In [2]:
import datetime

In [3]:
datetime.date(2023, 10, 9).isocalendar()[1]

41

In [5]:
 inital_week_nb = datetime.date(2023, 10, 9).isocalendar()[1]
current_week_nb = datetime.datetime.now().isocalendar()[1]

print("initial week nb: " + str(inital_week_nb))
print("current week nb: " + str(current_week_nb))

# increment index by 1 every 2 weeks
index = (current_week_nb - inital_week_nb) // 2



initial week nb: 41
current week nb: 25


In [6]:
index

-8

In [8]:
from apscheduler.schedulers.background import BackgroundScheduler
from apscheduler.triggers.cron import CronTrigger

In [11]:
scheduler = BackgroundScheduler()
scheduler.start()


trigger_roles = CronTrigger(
        year="*", month="*", day="1", hour="10", minute="0", second="0"
    )

In [12]:
trigger_roles

<CronTrigger (year='*', month='*', day='1', hour='10', minute='0', second='0', timezone='Europe/Berlin')>

In [91]:
import requests
import pandas as pd
import folium
import branca
import anytree

def get_en(field):
    try:
        return field['en']
    except (KeyError, TypeError):
        try:
            return field['fr']
        except (KeyError, TypeError):
                try:
                    return field['de']
                except (KeyError, TypeError):
                    return field
                

headers = {'Accept': 'application/json'}
base_url = 'https://www.zuerich.com'
data_url = base_url + '/en/api/v2/data'

In [115]:
def get_main_id(data_url):
    r = requests.get(data_url, headers=headers)
    data = r.json()
    main = [e for e in data if e['parent'] == '0']

    to_return = {}

    for element in main:
        to_return[element['name']['en']] = element['id']

    return to_return

def get_sub_main_id(id_url):
    r = requests.get(id_url, headers=headers)
    data = r.json()

    to_return = {}

    for element in data:
        print(element)
        #to_return[element['name']['en']] = element['id']

    return to_return



In [116]:
main = get_main_id(data_url=data_url)

In [117]:
get_sub_main_id(id_url=id_url)

{'@context': 'https://schema.org/', '@type': 'PerformingArtsTheater', '@customType': None, 'identifier': '1011143', 'copyrightHolder': {'de': 'Zürich Tourismus www.zuerich.com', 'en': 'Zurich Tourism www.zuerich.com', 'it': 'Zürich Tourismo www.zuerich.com', 'fr': 'Zürich Tourisme www.zuerich.com'}, 'license': 'BY-SA', 'tomasBookingId': None, 'category': {'Nightlife': {'swissId': ''}, 'Theater': {'swissId': ''}, 'Parties': {'swissId': ''}, 'Comedy': {'swissId': ''}}, 'name': {'de': 'Theatersaal Volkshaus Zürich', 'en': 'Theatersaal Volkshaus Zürich', 'fr': 'Theatersaal Volkshaus Zürich', 'it': 'Theatersaal Volkshaus Zürich'}, 'disambiguatingDescription': {'de': 'Direkt am Helvetiaplatz geniessen Musikbegeisterte, Kulturinteressierte und Partygänger:innen unvergessliche Momente in Zürich.', 'en': 'Directly on Helvetiaplatz, music lovers, culture enthusiasts, and partygoers can enjoy unforgettable moments in Zurich.', 'fr': 'Sur la place Helvetiaplatz, les mélomanes, les amateurs de cult

{}

In [88]:
id = id_list['Parties']
id_url = data_url +  f'?id={id}'

r = requests.get(id_url, headers=headers)
data = r.json()

#fr_data = [{k: get_en(v) for (k,v) in f.items()} for f in data]
df = pd.DataFrame(data)
df

,@context,@type,@customType,identifier,copyrightHolder,license,tomasBookingId,category,name,disambiguatingDescription,...,price,photo,dateModified,opens,openingHours,openingHoursSpecification,specialOpeningHoursSpecification,address,geoCoordinates,place
0,https://schema.org/,PerformingArtsTheater,None,1011143,"{'de': 'Zürich Tourismus www.zuerich.com', 'en...",BY-SA,None,"{'Nightlife': {'swissId': ''}, 'Theater': {'sw...","{'de': 'Theatersaal Volkshaus Zürich', 'en': '...",{'de': 'Direkt am Helvetiaplatz geniessen Musi...,...,"{'de': None, 'en': None, 'fr': None, 'it': None}",None,2024-06-20T16:21,[],,None,{'de': '<p>&Ouml;ffnungszeiten gem&auml;ss <a ...,"{'addressCountry': 'CH', 'addressLocality': 'Z...","{'latitude': 47.375577, 'longitude': 8.527125}",[]


In [63]:
from anytree import Node, RenderTree
from urllib.parse import urljoin

headers = {'Accept': 'application/json'}
base_url = 'https://www.zuerich.com'
data_url = base_url + '/en/api/v2/data'
r = requests.get(data_url, headers=headers)
data = r.json()
data

def find_children(data, parent):
    children = [e fqor e in data if e['parent'] == parent.id]
    for c in children:
        node = Node(id=c['id'], name=c['name'].get('en', c['name']), urlpath=c['path'], parent=parent)
        find_children(data, node)

root = Node(id='0', name="Root", urlpath="/data")
find_children(data, root)

# Zeige den Baum an
for pre, _, node in RenderTree(root):
    print("%s%s" % (pre, node.name))

Root
├── Accommodation
│   ├── Apartments
│   ├── Vacation Apartments
│   ├── Hostels
│   ├── B&Bs
│   ├── Campsites
│   └── Hotels
├── Events
│   ├── Other
│   ├── Exhibitions
│   ├── Children and families
│   ├── Classical
│   ├── Comedy
│   ├── Conferences and congresses
│   ├── Parties
│   ├── Festivals
│   ├── Trade Fair / Market
│   ├── Sports Events
│   ├── Opera
│   ├── Pop, Rock, Jazz
│   ├── Theater
│   └── Traditional
├── Gastronomy
│   ├── Nightlife
│   │   ├── Atmosphere
│   │   │   ├── Lively and Cheerful
│   │   │   ├── Alternative and Arty
│   │   │   ├── Cool and Trendy
│   │   │   ├── Cultural and Inspiring
│   │   │   ├── Relaxed and Cozy
│   │   │   ├── Glamorous and Chic
│   │   │   ├── LGBTQ+*
│   │   │   └── Traditional and Down-to-earth
│   │   ├── Bars & Lounges
│   │   │   ├── After-work
│   │   │   ├── Choice Spirits
│   │   │   ├── Cocktail Bar
│   │   │   ├── Music Bar / Live Music
│   │   │   ├── Wine Bar
│   │   │   ├── Neighborhood Bar
│   │   │   ├── Ho

In [74]:
children = [e for e in data if e['parent'] == '75']


In [ ]:
def get_restaurant_types():
    response = requests.get(url)
    data = response.json()
    # Supposons que les types de restaurants soient dans une clé 'restaurant_types'
    return data.get('restaurant_types', [])

# Fonction pour récupérer les restaurants en fonction du type sélectionné
def get_restaurants_by_type(restau_type):
    # Modifiez cette URL et les paramètres selon votre API réelle
    response = requests.get(url, params={'type': restau_type})
    data = response.json()
    return data.get('restaurants', [])

# Gestion de la commande /restau
@bot.message_handler(commands=['restau'])
def handle_restau(message):
    restau_types = get_restaurant_types()
    markup = types.InlineKeyboardMarkup()
    for restau_type in restau_types:
        markup.add(types.InlineKeyboardButton(text=restau_type, callback_data=restau_type))
    bot.send_message(message.chat.id, "Choisissez un type de restaurant:", reply_markup=markup)

# Gestion des réponses aux boutons cliquables
@bot.callback_query_handler(func=lambda call: True)
def callback_query(call):
    restau_type = call.data
    restaurants = get_restaurants_by_type(restau_type)
    response_text = f"Restaurants de type {restau_type}:\n\n"
    for restau in restaurants:
        response_text += f"{restau['name']}\nAdresse: {restau['address']}\n\n"
    bot.send_message(call.message.chat.id, response_text)